# PACKAGE

In [ ]:
!pip install python-docx


In [ ]:
!pip install docx2txt


In [ ]:
# !pip install --upgrade pyresparser

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
!python -m spacy download fr_core_news_md
!python -m spacy download en_core_web_md

In [ ]:
import spacy
fr_nlp = spacy.load("fr_core_news_md")
en_nlp = spacy.load("en_core_web_md")

In [ ]:
!pip install pdfminer.six

In [ ]:
!pip install unidecode

In [ ]:
!pip install fitz

In [ ]:
!pip install textacy
!pip install langid

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.download ('wordnet')


In [ ]:
import pandas as pd
from docx import Document
import csv
import numpy as np
import itertools
import os
import requests

In [ ]:
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# CLASS

## extraction


In [ ]:
from io import StringIO
import glob
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import docx2txt
import re


In [ ]:
import unicodedata

class extract_text:


    def pdf_extract_text (filename, pages=None):
          if not pages:
            pagenums=set()
          else:
            pagenums=set(pages)

          output= StringIO()
          # data={}
          contenue=[]
          # name=[]
          # for filename in glob.glob ('/content/drive/MyDrive/CV/*.pdf'):
          with open (filename, 'rb') as in_file:
            if filename.endswith ('.docx') or filename.endswith ('.doc'):
              text=docx2txt.process(in_file)
              text=unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8') 
              contenue= ' '.join([str(elem) for elem in (text.split('\n'))])
              text=unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8') 
            elif filename.endswith ('.pdf'):
              parser= PDFParser(in_file)
              doc=PDFDocument(parser)
              manager= PDFResourceManager()
              converter= TextConverter(manager, output, laparams=LAParams())
              interpreter= PDFPageInterpreter (manager, converter)
              for page in PDFPage.get_pages(in_file, pagenums):
                interpreter.process_page(page)
              text=output.getvalue()
              text= unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8') 
              contenue= ' '.join([str(elem) for elem in (text.split('\n'))])
              in_file.close()
              converter.close()
              output.close()
            else:
              pass
          return  contenue

    def extract_number(text):
          numbers=[]
          phone=(re.findall((r'[0-9]?[0-9 (\)]{8,15}[0-9]') ,str(text)))
          if phone:
                    number= ''.join(phone[0])
                    if text.find(number)>=0 and len(number)<20:
                      numbers.append(''.join(str(elem) for elem in number))
          return numbers

    def extract_date (text):
          dates=[]
          dates.append(re.findall((r"\d{1,2}[-/]\d{1,2}[-/]\d{2,4}") ,text))
          dates.append(re.findall((r"\w{4,9} ?\d{2,4}") ,text))
          dates.append(re.findall((r"\d{2,4}[ ][-/][ ]\d{2,4}") ,text))
          
          return dates


## process clean

In [ ]:
import unidecode

class cleaner:

    def __init__(self) -> None:
        pass
    
   
    def clean_text(sentence:str):
        
        text = cleaner.remove_special_char(cleaner.extract_only_text(sentence))
        return cleaner.remove_double_space(text)
    
    def remove_double_space(text):
         return re.sub(r'([\s])\1{1,}|([\.])\1{1,}', r'\1', text)
        #  return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8')

    def extract_only_text(text):
        # text=unidecode.unidecode(text).lower()
        text = re.sub('\S*www\S*\s?', '', str(text))
        text = re.sub('\S*http\S*\s?', '', str(text))
        # text = re.sub('(\d+\.)+\d*.*','',str(text))
        text = re.sub('\d+','',str(text))
        text = re.sub("-|_",'',str(text))
        text = re.sub("\n|\r",'',str(text))
        text = re.sub("[0-9]+",'',str(text))
        text= re.sub(r'\x00|\uf001','fi',str(text))
        text= re.sub(r'cv','',str(text))
        text= re.sub(r'\x0c','',str(text))
        text= re.sub(r'curriculum |vitae|curriculum Vitae','',str(text))
        # text= re.sub(r'[^www]?[A-Za-z] ','',text)
        text= re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+', '', str(text))
        # text= re.sub(r'www[a-z0-9\.\-+_]+\.[a-z]+[/-][a-zA-Z]?[a-z\-]+', r'', str(text))
        text= re.sub(r'\uf005|\uf006|\uf10b|\ue81e|\ue811|\ue80d|\ue81a|\ue803', r'', str(text))
      
        return text
    
    def remove_special_char(text:str):
        return re.sub(r"[^a-zA-Z0-9\-“”\s\'\.]"," ", text)

In [ ]:
from sys import path
from pathlib import Path

liste_test = glob.glob ('/content/drive/MyDrive/projet pfe/data/*')
liste=[]
title=[]
Email=[]
number=[]
for filename in liste_test:
        # text = pdf_extract_text (filename)
        text = extract_text.pdf_extract_text (filename).lower()
        texte=cleaner.clean_text(text)
        liste.append(texte)
        email= (re.findall((r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+'), str(text)))
        Email.append( ' '.join([str(elem) for elem in email]))
        number.append( extract_text.extract_number(text))
        # liste.append(text)
        title.append(Path(filename).stem)


data=pd.DataFrame(liste, index=None)
data['Candidates']= pd.DataFrame(title)
data['Emails']= pd.DataFrame(Email)
data['Phone_number']= pd.DataFrame(number)
data['Resumes']= pd.DataFrame(liste)

data= data.drop(columns=0)

data.to_csv('data.csv') 
  
df = pd.read_csv('data.csv', engine='python' ) 
df= df.drop(columns='Unnamed: 0')
df
        

In [ ]:
df=pd.read_csv("/content/data.csv")

In [ ]:
df

## evaluator



In [ ]:
!pip install git+https://github.com/LIAAD/yake 

In [ ]:
import yake
from langid.langid import LanguageIdentifier, model

class LangDetection:

    def __init__(self):
        self.text=None
        self.language=None

    def override(self, text):
        self.text = text
        identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
        self.language  = identifier.classify(self.text)


        return self.language




    def Detection(text):
        identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
        lang = identifier.classify(text)
        if "en" in lang:
                    doc= en_nlp(text)
                    language= 'en'
        elif "fr" in lang:
                    doc= fr_nlp(text)
                    language= 'fr'
        return language



class Keywords (LangDetection):

      def __init__(self,text):
          self.lang= LangDetection()
          

      def keys (text):
          language = LangDetection.Detection(text)
          max_ngram_size = 3
          deduplication_thresold = 0.9
          deduplication_algo = 'seqm'
          windowSize = 1
          numOfKeywords = 50

          custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
          keywords = custom_kw_extractor.extract_keywords(text)
          return [kw for kw in keywords]




In [ ]:
Keywords.keys(text=df["Resumes"][2])

In [ ]:
from langid.langid import LanguageIdentifier, model

def Document(text):
                  # text= extract_text.pdf_extract_text(text)
                  identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
                  lang = identifier.classify(text)
                  if "en" in lang:
                    doc= en_nlp(text)
                  elif "fr" in lang:
                    doc= fr_nlp(text)
                  return doc

def token (sentence):
  doc= Document(sentence)
  return [token.text for token in doc if (not token.is_stop) ]

def stopword (sentence):
  sentence = extract_text.pdf_extract_text(sentence).lower()
  sentence = cleaner.clean_text(sentence)
  doc= Document(sentence)
  return [token.lemma_ for token in token(sentence) if (not token.is_punct) ]


def token_sentence (sentence):
  doc= Document(sentence)
  return [token.text for token in doc.sents]

def ner (sentence):
  doc= Document (sentence)
  return [(x.text, x.label_) for x in doc.ents]






In [ ]:

df['Resum'] = df.Resumes.apply(lambda x: ' '.join(ner(x)))
df.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis

class Evaluator:

    def computeTFIDF(self, documents):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform(documents)
        feature_names = vectorizer.get_feature_names()
        dense = vectors.todense()
        denselist = dense.tolist()
        df = pd.DataFrame(denselist, columns=feature_names)
        df.index = ([f"TFIDF_{i}" for i in range(1,len(documents)+1)])

        return df.T 

    def similarity(self,documents):
        similarities = {}
        for i,document in enumerate(documents,1):
            doc = Document(document)
            sim = []
            for document_ in documents:
                sim.append(doc.similarity(Document(document_))) 
            similarities[f"Doc_{i}"] = sim
        df = pd.DataFrame(similarities, index=[f"Doc_{i}" for i in range(1,len(documents)+1)])
        return df

    def display_lda(self,lda_model, corpus, id2word):
        return
        pyLDAvis.enable_notebook()
        vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
        return vis

    def LDA(self,data_words):
        id2word = corpora.Dictionary(data_words)
        corpus = []
        
        for text in data_words:
            new = id2word.doc2bow(text)
            corpus.append(new)
        num_topic = len(corpus)
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topic,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           eta = [.01]*len(id2word.keys()),
                                           alpha=[.01]*num_topic)

        

        return lda_model ,self.display_lda(lda_model,corpus,id2word)
        #lda_model.print_topics()

    # def bert (sel, documents):


In [ ]:
test= glob.glob ("/content/drive/MyDrive/projet pfe/CV/chaima.pdf")

tok=[]
for filename in liste_test:
  print (evaluator.computeTFIDF)